## Effortlessly Transform Your Synchronous Function into a Multi-threaded function

Credit goes to comcrawl for the multi-threading function, which I extracted from their script: : 
https://github.com/michaelharms/comcrawl/blob/a89236080c5e7f4ce6a2e0d39c5f59671f22181e/comcrawl/utils/multithreading.py#L12

Imagine the simplicity of converting your synchronous function into a multi-threaded powerhouse with just a single function called `make_multithreaded` as below:

```python

def scrape(url):
    response = session.get(url)
    return response.json()

multithreaded_scrape = make_multithreaded(scrape, threads=10)
data_list = multithreaded_scrape(input_list)
print(data_list)

```


In [1]:
import requests
from typing import Callable
import concurrent.futures

def make_multithreaded(func: Callable,
                       threads: int) -> Callable:
    """Creates a multithreaded version of a function.

    Args:
        func: Function that is meant to be executed on
            a list of input objects.
        threads: The number of threads the multithreaded
            version of the function should use.

    Returns:
        A multithreaded version of the `func` input function.

    """

    def multithreaded_function(input_list: list, *args) -> list:
        """Executes function on input list using multiple threads.

        Args:
            input_list: The list of objects a function should be
                executed on.
            *args: Variable length argument list of additional
                parameters needed for the function to be executed.

        Returns:
            List of results after all input list elements were
            processed. Input order might not be preserved in
            output list.

        """
        results = []

        with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
            future_to_input_item = {
                executor.submit(
                    func,
                    input_item,
                    *args
                ): input_item for input_item in input_list
            }

            for future in concurrent.futures.as_completed(future_to_input_item):
                result = future.result()

                if isinstance(result, list):
                    results.extend(result)
                else:
                    results.append(result)

        return results

    return multithreaded_function


In [2]:
%%time

input_list = ["https://httpbin.org/delay/10"]*10

session = requests.Session()

def scrape(url):
    response = session.get(url)
    return response.json()

multithreaded_scrape = make_multithreaded(scrape, threads=10)
data_list = multithreaded_scrape(input_list)

print(data_list)

[{'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.31.0', 'X-Amzn-Trace-Id': 'Root=1-65ed801b-70e13ca7579d6ba84126a229'}, 'origin': '34.127.100.114', 'url': 'https://httpbin.org/delay/10'}, {'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.31.0', 'X-Amzn-Trace-Id': 'Root=1-65ed801b-5d918552206ce4ce677111c7'}, 'origin': '34.127.100.114', 'url': 'https://httpbin.org/delay/10'}, {'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.31.0', 'X-Amzn-Trace-Id': 'Root=1-65ed801b-18968c3233b5e2c82a94be9d'}, 'origin': '34.127.100.114', 'url': 'https://httpbin.org/delay/10'}, {'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept

In [3]:
import pandas

pandas.DataFrame(data_list)

,args,data,files,form,headers,origin,url
0,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10
1,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10
2,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10
3,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10
4,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10
5,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10
6,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10
7,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10
8,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10
9,{},,{},{},"{'Accept': '*/*', 'Accept-Encoding': 'gzip, de...",34.127.100.114,https://httpbin.org/delay/10


## Computing Environment